In [1]:
%pylab inline
import pandas as pd
import csv as csv
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from math import sqrt
from scipy import stats
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def load_data(filename, na_handler={}):
    df = pd.read_csv(filename, sep='\t', na_values={k:v for k, v in na_handler.iteritems() if isinstance(v,int)}, header=False)
    return df
    
def create_target_1tail(df, sample_and_size_cols=[], baseline_col='', col_name='target', p=0.05):
    df[col_name] = np.vectorize(poisson_p)(*sample_and_size_cols, cons=baseline_col, p_val=p)
    return col_name

def add_rate_col(df, event_col='', per_col='', col_name=''):
    df[col_name] = df[event_col]/df[per_col]
    return col_name

def add_lift_col(df, samp_col='', baseline_col='', col_name=''):
    df[col_name] = df[event_col]/df[per_col] - 1
    return col_name

def power_calc(imps = '', baseline_cvr = '', lift = ''):
    if imps == '':
        if baseline_cvr !='' and lift !='':
            imps = 4/((baseline_cvr)*((sqrt(lift)+1)**2))
            #print str(int(imps)) + ' or more impressions needed'
            try:
                return int(imps)
            except:
                return 10000000000

    elif baseline_cvr == '':
        if imps !='' and lift !='':
            baseline_cvr = 4/((imps)*((sqrt(lift)+1)**2))
            print str(baseline_cvr) + ' or higher baseline conversion rate needed'
            return baseline_cvr

    elif lift == '':
        if baseline_cvr !='' and imps !='':
            lift = (sqrt(4/(baseline_cvr*imps))-1)**2
            print 'Can detect '+ str(int(lift)) + 'x lift'
            return int(lift)
    else:
        imps_needed = 4/((baseline_cvr)*((sqrt(lift)+1)**2)) - imps
        print 'Need ' + str(int(imps_needed)) + ' more impressions to reach significance'
        return int(imps_needed)

def poisson_p(line_cvr, imps, cons, p_val=0.05):
    # mu: expected number of conversions
    # k: observed number of conversions (list)
    # returns the p-value (probability that we'd observe k or greater given H0)
    # note: if no conversions have occurred, poisson is bounded by 3/n trials

    #if there are no conversions but also too few impressions, do not use this record
    imps_needed = power_calc(baseline_cvr = line_cvr, lift = 1000)
    if imps_needed > imps: return -1

    p = stats.poisson.sf(cons-1,line_cvr*imps)

    if p < p_val: return 1
    else: return 0



In [ ]:
def percentile_by_group(row, group='advertiser_id', value=''):
    dist = df_test[df_test[group] == row[group]][value]
    value = row[value]
    return stats.percentileofscore(dist, value)

'''df_test['ol_percentile']= df_test.apply(lambda row: np.round(percentile_by_group(row, group='advertiser_id', value='pct_overlap'),2), axis=1)
df_test['hl_percentile']= df_test.apply(lambda row: np.round(percentile_by_group(row, group='line_id', value='hist_lift'),2), axis=1)
df_test['wr_percentile']= df_test.apply(lambda row: np.round(percentile_by_group(row, group='line_id', value='win_rate'),2), axis=1)
df_test['pi_percentile']= df_test.apply(lambda row: np.round(percentile_by_group(row, group='advertiser_id', value='popularity_index'),2), axis=1)
df_test['bp_percentile']= df_test.apply(lambda row: np.round(percentile_by_group(row, group='advertiser_id', value='avg_bid_price'),2), axis=1)'''